In [ ]:
!pip install -U transformers datasets evaluate --force-reinstall

  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
  Using cached datasets-4.4.1-py3-none-any.whl.metadata (19 kB)
  Using cached evaluate-0.4.6-py3-none-any.whl.metadata (9.5 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached numpy-2.3.4-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached packaging-25.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached pyyaml-6.0.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (2.4 kB)
  Using cached regex-2025.11.3-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (40 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached safetensors-0.6.2-cp38-abi3-manylinux_2_17_x86_64.

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.3 MB/s eta 0:00:00


In [ ]:
import os
import re
import random
from pathlib import Path
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from datasets import Dataset, DatasetDict
from evaluate import load as load_metric
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    set_seed
)

In [ ]:

# ----------------------------
# Settings (tweak these)
# ----------------------------
SEED = 42
MODEL_NAME = "bert-base-uncased"
MAX_LENGTH = 128
BATCH_SIZE = 8             # per device
EPOCHS = 2
OUTPUT_DIR = "./bert_sentiment_results"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
USE_SUBSAMPLE = True       # set True to develop quickly on a small subset
SUBSAMPLE_SIZE = 50000    # if USE_SUBSAMPLE True
# If using kagglehub, you can programmatically download dataset and set CSV_PATH to actual path

os.makedirs(OUTPUT_DIR, exist_ok=True)
set_seed(SEED)

print("Device:", DEVICE)


Device: cuda


In [ ]:

import kagglehub
from kagglehub import KaggleDatasetAdapter
# ----------------------------
# 1) Load CSV (handles encoding)
# ----------------------------
def load_sentiment140(csv_path):
    # CSV has no header; columns: target, ids, date, flag, user, text
    df = pd.read_csv(csv_path, encoding='ISO-8859-1', header=None,
                     names=['target','ids','date','flag','user','text'])
    # Keep only text + label
    df = df[['target','text']].copy()
    # Map label: 0 -> negative, 4 -> positive -> convert 4 to 1
    df['target'] = df['target'].replace(4, 1)
    # Optional: drop NaNs
    df.dropna(subset=['text','target'], inplace=True)
    # Shuffle
    df = df.sample(frac=1, random_state=SEED).reset_index(drop=True)
    return df

print("Loading CSV...")
file_path = "training.1600000.processed.noemoticon.csv"
dataset_path = kagglehub.dataset_download("kazanova/sentiment140")
# 🔧 Read CSV manually with encoding
CSV_PATH=f"{dataset_path}/{file_path}"
df = load_sentiment140(CSV_PATH)
print("Loaded rows:", len(df))
print(df['target'].value_counts())

Loading CSV...
Using Colab cache for faster access to the 'sentiment140' dataset.
Loaded rows: 1600000
target
0    800000
1    800000
Name: count, dtype: int64


In [ ]:
# ----------------------------
# 1b) Optional subsample for dev
# ----------------------------
if USE_SUBSAMPLE:
    df = df.sample(n=SUBSAMPLE_SIZE, random_state=SEED).reset_index(drop=True)
    print("Subsampled rows:", len(df))

Subsampled rows: 50000


In [ ]:
# ----------------------------
# 2) Basic cleaning function
# ----------------------------
def clean_text(s):
    # basic cleaning: remove URLs, mentions, html entities, extra whitespace
    s = str(s)
    s = re.sub(r"http\S+|www\.\S+", "", s)
    s = re.sub(r"@\w+", "", s)
    s = re.sub(r"&amp;", "&", s)
    s = re.sub(r"[^0-9A-Za-z\s#']", " ", s)  # keep hashtags and apostrophes if you want
    s = re.sub(r"\s+", " ", s).strip()
    return s

print("Cleaning text (this may take a while)...")
df['text'] = df['text'].map(clean_text)


Cleaning text (this may take a while)...


In [ ]:
# ----------------------------
# 3) Train / Val / Test split
# ----------------------------
train_df, test_df = train_test_split(df, test_size=0.1, random_state=SEED, stratify=df['target'])
train_df, val_df = train_test_split(train_df, test_size=0.1111, random_state=SEED, stratify=train_df['target'])
# results ~ 80/10/10
print("train/val/test:", len(train_df), len(val_df), len(test_df))

# Convert to HF Dataset
train_ds = Dataset.from_pandas(train_df[['text','target']])
val_ds   = Dataset.from_pandas(val_df[['text','target']])
test_ds  = Dataset.from_pandas(test_df[['text','target']])
dataset = DatasetDict({"train": train_ds, "validation": val_ds, "test": test_ds})
print(dataset)

train/val/test: 40000 5000 5000
DatasetDict({
    train: Dataset({
        features: ['text', 'target', '__index_level_0__'],
        num_rows: 40000
    })
    validation: Dataset({
        features: ['text', 'target', '__index_level_0__'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['text', 'target', '__index_level_0__'],
        num_rows: 5000
    })
})


In [ ]:
# ----------------------------
# 4) Tokenizer + encode
# ----------------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def preprocess_function(examples):
    # examples['text'] is a list
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=MAX_LENGTH)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:

# Use batched mapping
print("Tokenizing dataset (batched)...")
tokenized = dataset.map(preprocess_function, batched=True, remove_columns=['text'])

# tokenized has input_ids, attention_mask, target
tokenized = tokenized.rename_column("target", "labels")
tokenized.set_format("torch")
print(tokenized)

Tokenizing dataset (batched)...


Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 40000
    })
    validation: Dataset({
        features: ['labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 5000
    })
})


In [ ]:
# ----------------------------
# 5) Model init
# ----------------------------
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
model.to(DEVICE)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:

# ----------------------------
# 6) Metrics
# ----------------------------
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
# ----------------------------
# 7) TrainingArguments + Trainer
# ----------------------------
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    weight_decay=0.01,
    num_train_epochs=EPOCHS,
    logging_dir=f"{OUTPUT_DIR}/logs",
    logging_strategy="steps",
    logging_steps=200,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    fp16=torch.cuda.is_available(),     # use mixed precision if available
    save_total_limit=3
)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


/tmp/ipython-input-2352651356.py:25: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


API_KEY = ***f95365e980c47da5e35cd78b81f542b4f9ba2b07***

In [ ]:
!pip uninstall -y wandb
import os
os.environ["WANDB_DISABLED"] = "true"

Found existing installation: wandb 0.22.3
Uninstalling wandb-0.22.3:
  Successfully uninstalled wandb-0.22.3


In [ ]:
# ----------------------------
# 8) Train
# ----------------------------
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "disabled"
os.environ["HF_DISABLE_WANDB"] = "1"
!pip uninstall -y wandb




print("Starting training...")
trainer.train()
print("Training finished.")

Starting training...


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.396200,0.421748,0.816200,0.814274,0.821727,0.817984
2,0.308200,0.555926,0.829000,0.832932,0.825308,0.829103


Training finished.


In [ ]:
# ----------------------------
# 9) Evaluate on test set
# ----------------------------
print("Evaluating on test set...")
metrics = trainer.evaluate(tokenized["test"])
print("Test metrics:", metrics)

Evaluating on test set...


Test metrics: {'eval_loss': 0.5420933961868286, 'eval_accuracy': 0.8322, 'eval_precision': 0.837772397094431, 'eval_recall': 0.8261042578591326, 'eval_f1': 0.8318974153476257, 'eval_runtime': 12.0779, 'eval_samples_per_second': 413.979, 'eval_steps_per_second': 51.747, 'epoch': 2.0}


In [ ]:
# Confusion matrix (optional)
preds_output = trainer.predict(tokenized["test"])
preds = np.argmax(preds_output.predictions, axis=-1)
cm = confusion_matrix(tokenized["test"]["labels"], preds)
print("Confusion Matrix:\n", cm)

Confusion Matrix:
 [[2085  402]
 [ 437 2076]]


In [ ]:
# ----------------------------
# 10) Save model & tokenizer
# ----------------------------
print("Saving model and tokenizer...")
trainer.save_model(OUTPUT_DIR)        # saves model config + weights
tokenizer.save_pretrained(OUTPUT_DIR)
print("Saved to", OUTPUT_DIR)

Saving model and tokenizer...
Saved to ./bert_sentiment_results


In [ ]:
# ----------------------------
# 11) Quick inference example
# ----------------------------
def predict_text(text):
    model.eval()
    inputs = tokenizer(clean_text(text), return_tensors="pt", truncation=True, padding=True, max_length=MAX_LENGTH).to(DEVICE)
    with torch.no_grad():
        logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1).cpu().item()
    return "positive" if pred == 1 else "negative"

print("Sample:", predict_text("I love this phone, battery lasts forever!"))
print("Sample:", predict_text("one of the worst"))

Sample: positive
Sample: negative


In [ ]:
!zip -r model.zip bert_sentiment_results

  adding: bert_sentiment_results/ (stored 0%)
  adding: bert_sentiment_results/tokenizer.json (deflated 71%)
  adding: bert_sentiment_results/special_tokens_map.json (deflated 42%)
  adding: bert_sentiment_results/model.safetensors (deflated 7%)
  adding: bert_sentiment_results/config.json (deflated 49%)
  adding: bert_sentiment_results/logs/ (stored 0%)
  adding: bert_sentiment_results/logs/events.out.tfevents.1763136225.107606c07076.307.1 (deflated 30%)
  adding: bert_sentiment_results/logs/events.out.tfevents.1763134728.107606c07076.307.0 (deflated 64%)
  adding: bert_sentiment_results/checkpoint-10000/ (stored 0%)
  adding: bert_sentiment_results/checkpoint-10000/rng_state.pth (deflated 26%)
  adding: bert_sentiment_results/checkpoint-10000/trainer_state.json (deflated 79%)
  adding: bert_sentiment_results/checkpoint-10000/tokenizer.json (deflated 71%)
  adding: bert_sentiment_results/checkpoint-10000/optimizer.pt (deflated 20%)
  adding: bert_sentiment_results/checkpoint-10000/sca

In [ ]:
from google.colab import files
files.download("model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_id = "girizhhh/sentimentAnalyzer"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(model_id)

text = ""
inputs = tokenizer(text, return_tensors="pt")

outputs = model(**inputs)
print(outputs.logits)

tensor([[-0.4220,  0.4597]], grad_fn=<AddmmBackward0>)
